# Limpieza y Tokenizacion

- Cargamos librerias

In [1]:
import pandas as pd
import re
import json

In [9]:
#import spacy
#sp = spacy.load('es_core_news_sm')
#stop_words = sp.Defaults.stop_words
#from nltk.corpus import stopwords
#stopwords_español = stopwords.words('spanish')
#aditionalwords= [palabra for palabra in stopwords_español if palabra not in stop_words ]
#stop_words.extend(aditionalwords)

In [3]:
#Guardar Stop Words
with open('./Data/stopwords.txt', 'w') as f:
    f.write(json.dumps(stop_words))

NameError: name 'stop_words' is not defined

### Cargamos Stopwords

In [2]:
#Leer Stop words como set
with open('./Data/stopwords.txt', 'r') as f:
    stopwords = json.loads(f.read())
len(stopwords)


704

# Funciones Limpieza de Texto
- Estandarizar tildes
- Revisar abreviaturas

In [3]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words             containing numbers.'''
    #text = fix_encoding(text)    
    #text = re.sub('[‘’“”…«».]', '', text)
    #text = re.sub('[\n\t]', ' ', text)
    #text = re.sub("(\\d|\\W)+", ' ', text)  # Removing special characters and digits    
    #text = text.replace("_","")

    text = text.lower()
    text = re.sub("\d", '',text)
    pattern = re.compile(r"""                  # Flag para iniciar el modo verbose
              #(?:[A-Za-z]\.|\'|[A-Za-z])+            # Hace match con abreviaciones como U.S.A.Nombre's
              (?:[A-Za-z]\.)+            # Hace match con abreviaciones como U.S.A.        
              | \w+(?:-\w+)*         # Hace match con palabras que pueden tener un guión interno
              # \$?\d+(?:\.\d+)?%?  # Hace match con dinero o porcentajes como $15.5 o 100%
              # \.\.\.              # Hace match con puntos suspensivos
              # [][.,;"'?():-_`]    # Hace match con signos de puntuación
              """, re.X)

    result = pattern.findall(text)
    #nltk.regexp_tokenize(text, pattern)
    return result
    
#text = "En los E.U. U.S.A. esa postal vale $15.50... (feo) `hola` Acoso Escolar (Bullying) en San Juan de Pasto.... J.D.A.A.S. baby Santiago's clubc slsda-sdasd-ss D.I.A. A.C.E. dia"

#clean_text(text)


- Cargamos Datos Originales

In [4]:
resumenes_docentes = pd.read_excel("./Data/Resumenes.xlsx",sheet_name = "Proyectos Docentes")
resumenes_estudiantes = pd.read_excel("./Data/Resumenes.xlsx",sheet_name="studiantiles y Trabajos de G")

#Eliminamos  Columnas
resumenes_docentes.drop(columns=['No.'], inplace=True)
resumenes_estudiantes.drop(columns=['No.'], inplace=True)

#Eliminamos NAN
resumenes_docentes.dropna(inplace=True)
resumenes_estudiantes.dropna(inplace=True)

#Asignamos index
resumenes_docentes['index'] = [*range(1,len(resumenes_docentes)+1)]
resumenes_docentes.set_index('index', inplace=True)

resumenes_estudiantes['index'] = [*range(1,len(resumenes_estudiantes)+1)]
resumenes_estudiantes.set_index('index', inplace=True)

#Asignamos Columnas
resumenes_docentes.columns = ["codigo", "titulo", "resumen", "estado", "id_autor", "nombre_autor", 
                     "programa", "facultad", "convocatoria", "grupo_investigacion", "linea_investigacion", "palabras_clave"]

resumenes_estudiantes.columns = ['codigo', 'titulo', 'resumen', 'estado', 'id_autor', 'nombre_autor',
       'programa', 'departamento', 'facultad', 'nombre_asesor', 'convocatoria', 'grupo_investigacion', 'linea_investigacion', 'palabras_clave']


#Eliminamos registros no encontrados
resumenes_docentes['palabras_clave'] = resumenes_docentes['palabras_clave'].apply(lambda row: row if (row != 'No se encontraron palabras clave registradas') else " ")
resumenes_estudiantes['palabras_clave'] = resumenes_estudiantes['palabras_clave'].apply(lambda row: row if (row != 'No se encontraron palabras clave registradas') else " ")

### Corpus agregado al DF

In [5]:
#Agregamos Columna de Corpus palabras tokenizadas
resumenes_docentes['corpus'] = resumenes_docentes.apply(lambda row: row['titulo']+' '+row['resumen']+' '+' '+row['palabras_clave']+' '+row['nombre_autor']+' '+row['programa']+' '+' '+row['facultad']+' '+row['grupo_investigacion']+' '+row['linea_investigacion'],axis=1)

resumenes_estudiantes['corpus'] = resumenes_estudiantes.apply(lambda row: row['titulo']+' '+row['resumen']+' '+' '+row['palabras_clave']+' '+row['nombre_autor']+' '+row['nombre_asesor']+' '+row['programa']+' '+row['departamento']+' '+row['facultad']+' '+row['grupo_investigacion']+' '+row['linea_investigacion'],axis=1)

### Palabras Tokenizadas y Limpias

In [6]:
resumenes_docentes['palabras']=''
for index, row in resumenes_docentes.iterrows():
    tokens_corpus = clean_text(row['corpus'])
    palabras = [palabra for palabra in tokens_corpus if not palabra in stopwords and len(palabra) > 3]
    resumenes_docentes.at[index, 'palabras'] = palabras

resumenes_estudiantes['palabras']=''
for index, row in resumenes_estudiantes.iterrows():
    tokens_corpus = clean_text(row['corpus'])
    palabras = [palabra for palabra in tokens_corpus if not palabra in stopwords and len(palabra) > 3]
    resumenes_estudiantes.at[index, 'palabras'] = palabras

### Vocabulario del Resumen

In [7]:
resumenes_docentes['vocabulario'] = ''
resumenes_docentes['vocabulario_corpus'] = ' '
for index, row in resumenes_docentes.iterrows():
    vocabulario = set(row['palabras'])
    vocabulario_corpus = ' '.join([palabra for palabra in vocabulario])
    resumenes_docentes.at[index, 'vocabulario'] = vocabulario
    resumenes_docentes.at[index, 'vocabulario_corpus'] = vocabulario_corpus

resumenes_estudiantes['vocabulario'] = ''
resumenes_estudiantes['vocabulario_corpus'] = ''
for index, row in resumenes_estudiantes.iterrows():
    vocabulario = set(row['palabras'])
    vocabulario_corpus = ' '.join([palabra for palabra in vocabulario])
    resumenes_estudiantes.at[index, 'vocabulario'] = vocabulario
    resumenes_estudiantes.at[index, 'vocabulario_corpus'] = vocabulario_corpus


In [8]:
# Revisamos Datos
resumenes_docentes.head(3)

,codigo,titulo,resumen,estado,id_autor,nombre_autor,programa,facultad,convocatoria,grupo_investigacion,linea_investigacion,palabras_clave,corpus,palabras,vocabulario,vocabulario_corpus
index,,,,,,,,,,,,,,,,
1,1464.0,"""Diagnóstico del impacto de las iniciativas em...",El emprendimiento es entre otras acepciones es...,Vencido,13515.0,Francisco Rafael Ayala,Diseño,Artes,Docente 2017,Currículo y Universidad,Curriculos pertinentes,Diagnóstico,"""Diagnóstico del impacto de las iniciativas em...","[diagnóstico, impacto, iniciativas, emprendedo...","{dispuesto, profesionales, identificar, papel,...",dispuesto profesionales identificar papel bene...
2,1367.0,Acoso Escolar (Bullying) en San Juan de Pasto....,"A nivel mundial, uno de cada tres escolares ha...",Prorroga,11902.0,Harvey Mauricio Herrera Lopez,Psicología,Ciencias Humanas,Docente 2017,Psicología y Salud,Aspectos Psicosociales en Procesos de Salud,Acoso Escolar,Acoso Escolar (Bullying) en San Juan de Pasto....,"[acoso, escolar, bullying, juan, pasto, modelo...","{ítem, mental, representa, bullying, modelos, ...",ítem mental representa bullying modelos empíri...
3,1343.0,ACTIVIDAD ANTIBACTERIAL DE POLIFENOLES DEL AGU...,El cáncer gástrico (CG) es la principal causa ...,Vencido,10081.0,Nelson Humberto Hurtado Gutierrez,Química,Ciencias Exactas y Naturales,Docente 2016,GRUPO DE INVESTIGACIÓN EN PRODUCTOS DE IMPORTA...,Modelado molecular,cancer gastrico,ACTIVIDAD ANTIBACTERIAL DE POLIFENOLES DEL AGU...,"[actividad, antibacterial, polifenoles, aguaca...","{altas, frente, papel, dieta, preponderante, n...",altas frente papel dieta preponderante natural...


In [9]:

resumenes_estudiantes.head(3)

,codigo,titulo,resumen,estado,id_autor,nombre_autor,programa,departamento,facultad,nombre_asesor,convocatoria,grupo_investigacion,linea_investigacion,palabras_clave,corpus,palabras,vocabulario,vocabulario_corpus
index,,,,,,,,,,,,,,,,,,
1,741.0,"AISLAMIENTO, CARACTERIZACION, ESTABILIDAD Y AC...",El color es una de las cualidades sensoriales ...,Terminado,12324.0,Omaira Carolina Betancourt Ramos,Química,Química,Ciencias Exactas y Naturales,Nelson Humberto,Adicional de Trabajos de Grado 2013,GRUPO DE INVESTIGACIÓN EN PRODUCTOS DE IMPORTA...,Productos naturales,,"AISLAMIENTO, CARACTERIZACION, ESTABILIDAD Y AC...","[aislamiento, caracterizacion, estabilidad, ac...","{asocia, identificar, frente, picrylhydrazil, ...",asocia identificar frente picrylhydrazil añadi...
2,760.0,ANÁLISIS DE ALGORITMOS PARALELOS PARA LA TAREA...,La propuesta de la investigación se centra en ...,Cancelado,12479.0,Rosa Maria Zambrano Burbano,Ingeniería de Sistemas,Sistemas,Ingeniería,Manuel Ernesto,Adicional de Trabajos de Grado 2013,GRIAS,Deteccione de patrones con tecnicas de mineria...,,ANÁLISIS DE ALGORITMOS PARALELOS PARA LA TAREA...,"[análisis, algoritmos, paralelos, tarea, miner...","{identificar, sustraer, conoce, costos, razón,...",identificar sustraer conoce costos razón toma ...
3,1412.0,ANÁLISIS DE FUNCIONALIDAD DE LA HERRAMIENTA DE...,Hoy en día se puede encontrar muchas opciones ...,Vencido,13872.0,Carlos Mario Coral Cabrera,Ingeniería de Sistemas,Sistemas,Ingeniería,Silvio Ricardo,Estudiantil 2017,GRIAS,Descubrimiento de conocimiento en bases de datos,Análisis de funcionalidad,ANÁLISIS DE FUNCIONALIDAD DE LA HERRAMIENTA DE...,"[análisis, funcionalidad, herramienta, intelig...","{profesionales, libre, toma, presente, permite...",profesionales libre toma presente permite ayud...


# Experimental

In [23]:
resumenes_docentes.to_json(r'./Data/ResumenesDocentes.json')
resumenes_estudiantes.to_json(r'./Data/ResumenesEstudiante.json')


- Guardar CSV

In [124]:
resumenes_docentes.to_csv(r'./Data/ResumenesDocentes.csv')
resumenes_estudiantes.to_csv(r'./Data/ResumenesEstudiantes.csv')

In [24]:
json_resumen['lemas'] = ''
for index, row in json_resumen.iterrows():
    if( index > 1): break
    vocabulario = row['vocabulario']
    json_resumen.at[index, 'lemas'] = [lemmatizer(palabra) for palabra in vocabulario]
    print(f'Lemas Completo Resumen {index}')

Lemas Completo Resumen 1


# Carga de datos de prueba limpios

**ejecutar a partir de aqui para los LEMAS**

# Lematizar Palabras

### Cargar Datos ya procesados

In [ ]:
import pandas as pd
import ast

resumenes_docentes = pd.read_csv("./Data/ResumenesDocentes.csv", index_col=0)
resumenes_estudiantes = pd.read_csv("./Data/ResumenesEstudiantes.csv", index_col=0)


In [36]:
resumenes_docentes.head(1)

,codigo,titulo,resumen,estado,id_autor,nombre_autor,programa,facultad,convocatoria,grupo_investigacion,linea_investigacion,palabras_clave,corpus,palabras,vocabulario,vocabulario_corpus,lemas_corpus,lemas
index,,,,,,,,,,,,,,,,,,
1,1464.0,"""Diagnóstico del impacto de las iniciativas em...",El emprendimiento es entre otras acepciones es...,Vencido,13515.0,Francisco Rafael Ayala,Diseño,Artes,Docente 2017,Currículo y Universidad,Curriculos pertinentes,Diagnóstico,"""Diagnóstico del impacto de las iniciativas em...","['diagnóstico', 'impacto', 'iniciativas', 'emp...","{'dispuesto', 'profesionales', 'identificar', ...",dispuesto profesionales identificar papel bene...,disponer profesional identificar papel benefi...,"[disponer, profesional, identificar, papel, be..."


## Funcion de Lematizar una palabra

In [114]:
#Importar spacy para lemas
import spacy
nlp = spacy.load('es_core_news_sm')

#Eliminacion de Tildes y dialisis
normalizar = str.maketrans('áéíóúü','aeiouu')
#Diccionario lemas
diccionario_lemas = dict()

def lemmatizer(text):
  #Key Lema
  #Value las Palabras Originales
  lemas = ''
  doc = nlp(text)
  for word in doc:
    #Agregamos el corpus de lemas
    lemas = lemas + ' ' + word.lemma_
    lemas = lemas.translate(normalizar)

    #Eliminamos tildes de palabras
    cleaned_lema = word.lemma_.translate(normalizar)
    cleaned_word = word.text.translate(normalizar)

    #Obtenemos Lista de palabras asociadas al lema
    list_lemas = diccionario_lemas.get(cleaned_lema)

    #Verificar si el Lema existe
    if( list_lemas == None):
      #Agregamos palabra original y lema
      diccionario_lemas[cleaned_lema] = [ cleaned_word, cleaned_lema]
    else:
      #Revisamos si es una nueva palabra
      if cleaned_word not in list_lemas:
        diccionario_lemas[cleaned_lema].append(cleaned_word)

    #print(' '.join([word.lema_ for word in doc]))
  #print(lemas)
  return lemas

### Agregar Lemas a Docentes

### Lemas como corpus - DOCENTES

In [115]:
resumenes_docentes['lemas_corpus'] = ''
resumenes_docentes['lemas'] = ''
for index, row in resumenes_docentes.iterrows():
    #if( index > 1): break
    vocabulario = ast.literal_eval(row['vocabulario'])
    lemas_corpus = lemmatizer(row['vocabulario_corpus'])
    #lemas_corpus = lemas_corpus.replace(' .', '.') # espacios de abreviaturas

    lemas = list(lemas_corpus.split())
    resumenes_docentes.at[index, 'lemas_corpus'] = lemas_corpus
    resumenes_docentes.at[index, 'lemas'] = lemas    
    print(f'Lemas Completo Resumen {index}')


Lemas Completo Resumen 1
Lemas Completo Resumen 2
Lemas Completo Resumen 3
Lemas Completo Resumen 4
Lemas Completo Resumen 5
Lemas Completo Resumen 6
Lemas Completo Resumen 7
Lemas Completo Resumen 8
Lemas Completo Resumen 9
Lemas Completo Resumen 10
Lemas Completo Resumen 11
Lemas Completo Resumen 12
Lemas Completo Resumen 13
Lemas Completo Resumen 14
Lemas Completo Resumen 15
Lemas Completo Resumen 16
Lemas Completo Resumen 17
Lemas Completo Resumen 18
Lemas Completo Resumen 19
Lemas Completo Resumen 20
Lemas Completo Resumen 21
Lemas Completo Resumen 22
Lemas Completo Resumen 23
Lemas Completo Resumen 24
Lemas Completo Resumen 25
Lemas Completo Resumen 26
Lemas Completo Resumen 27
Lemas Completo Resumen 28
Lemas Completo Resumen 29
Lemas Completo Resumen 30
Lemas Completo Resumen 31
Lemas Completo Resumen 32
Lemas Completo Resumen 33
Lemas Completo Resumen 34
Lemas Completo Resumen 35
Lemas Completo Resumen 36
Lemas Completo Resumen 37
Lemas Completo Resumen 38
Lemas Completo Resume

### Lemas como corpus - ESTUDIANTES

In [122]:
resumenes_estudiantes['lemas_corpus'] = ''
resumenes_estudiantes['lemas'] = ''
for index, row in resumenes_estudiantes.iterrows():
    #if( index > 1): break
    vocabulario = ast.literal_eval(row['vocabulario'])
    lemas_corpus = lemmatizer(row['vocabulario_corpus'])
    lemas = list(lemas_corpus.split())
    resumenes_estudiantes.at[index, 'lemas_corpus'] = lemas_corpus
    resumenes_estudiantes.at[index, 'lemas'] = lemas    
    print(f'Lemas Completo Resumen {index}')


Lemas Completo Resumen 1
Lemas Completo Resumen 2
Lemas Completo Resumen 3
Lemas Completo Resumen 4
Lemas Completo Resumen 5
Lemas Completo Resumen 6
Lemas Completo Resumen 7
Lemas Completo Resumen 8
Lemas Completo Resumen 9
Lemas Completo Resumen 10
Lemas Completo Resumen 11
Lemas Completo Resumen 12
Lemas Completo Resumen 13
Lemas Completo Resumen 14
Lemas Completo Resumen 15
Lemas Completo Resumen 16
Lemas Completo Resumen 17
Lemas Completo Resumen 18
Lemas Completo Resumen 19
Lemas Completo Resumen 20
Lemas Completo Resumen 21
Lemas Completo Resumen 22
Lemas Completo Resumen 23
Lemas Completo Resumen 24
Lemas Completo Resumen 25
Lemas Completo Resumen 26
Lemas Completo Resumen 27
Lemas Completo Resumen 28
Lemas Completo Resumen 29
Lemas Completo Resumen 30
Lemas Completo Resumen 31
Lemas Completo Resumen 32
Lemas Completo Resumen 33
Lemas Completo Resumen 34
Lemas Completo Resumen 35
Lemas Completo Resumen 36
Lemas Completo Resumen 37
Lemas Completo Resumen 38
Lemas Completo Resume

### Guardamos diccionario de Lemas

In [123]:
#Guardamos diccionario como JSON
import json

with open('./Data/diccionario-lemas.json', 'w') as f:
    json.dump(diccionario_lemas, f)

In [86]:
#Lectura de lemas 
import json
with open('./Data/otro.json', 'r') as f:
    diccionario_lemas = json.loads(f.read())
diccionario_lemas

{'disponer': ['dispuesto', 'disponer'],
 'profesional': ['profesionales', 'profesional'],
 'identificar': ['identificar', 'identificar'],
 'papel': ['papel', 'papel'],
 'beneficio': ['beneficios', 'beneficio'],
 'flexible': ['flexible', 'flexible'],
 'pretender': ['pretende', 'pretender'],
 'conveniente': ['conveniente', 'conveniente'],
 'permitir': ['permitira', 'permitir'],
 'aporte': ['aportes', 'aporte'],
 'basico': ['basica', 'basico'],
 'relacionar': ['relaciones', 'relacionar'],
 'formador': ['formadores', 'formador'],
 'estudio': ['estudios', 'estudio'],
 'gradar': ['grado', 'gradar'],
 'nivel': ['nivel', 'nivel'],
 'complejidad': ['complejidad', 'complejidad'],
 'grupo': ['grupos', 'grupo'],
 'continuo': ['continuo', 'continuo'],
 'fomentar': ['fomentar', 'fomentar', 'fomento'],
 'tecnologico': ['tecnologicos', 'tecnologico'],
 'publico': ['publica', 'publico'],
 'beneficiar': ['beneficio', 'beneficiar'],
 'colectivo': ['colectivo', 'colectivo'],
 'creativo': ['creativo', 'cre

In [94]:
len(diccionario_lemas)

9208

### Agregar Lemas a Estudiantes

In [23]:
lemas = [ lemmatizer(palabra) for palabra in vocabulario]